<a href="https://colab.research.google.com/github/HalgasAdrian/LLM-For-Automatic-Hardware-Testbench-Generation/blob/DataPipeLineOptimization/notebooks/train_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Testbench Generation Training on Colab This notebook trains the model on Google Colab with GPU acceleration.

In [1]:
# Check GPU availability
!nvidia-smi

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Wed Aug 13 01:47:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P0             47W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 1. Setup Environment

## 3. Upload and Extract Project Files

In [2]:
# Option 1: Upload from local computer
from google.colab import files
uploaded = files.upload()  # Select llm_testbench_colab.zip

# Extract
!unzip -q llm_testbench_colab.zip
!ls -la

Saving llm_testbench_colab.zip to llm_testbench_colab.zip
total 200
drwxr-xr-x 1 root root   4096 Aug 13 01:48 .
drwxr-xr-x 1 root root   4096 Aug 13 01:41 ..
drwxr-xr-x 4 root root   4096 Aug 11 13:35 .config
drwxr-xr-x 2 root root   4096 Aug 13 01:48 configs
drwxr-xr-x 3 root root   4096 Aug 13 01:48 data
-rw-r--r-- 1 root root    177 Aug 10 23:07 .env
-rw-r--r-- 1 root root 152773 Aug 13 01:48 llm_testbench_colab.zip
drwxr-xr-x 2 root root   4096 Aug 13 01:48 notebooks
-rw-r--r-- 1 root root    159 Aug 12 21:47 requirements_colab.txt
-rw-r--r-- 1 root root    736 Aug 10 23:33 requirements.txt
drwxr-xr-x 1 root root   4096 Aug 11 13:36 sample_data
drwxr-xr-x 2 root root   4096 Aug 13 01:48 scripts
drwxr-xr-x 3 root root   4096 Aug 13 01:48 utils


## 4. Install Dependencies

In [3]:
# Install requirements
!pip install -q -r requirements_colab.txt

# Verify installations
!pip list | grep -E "torch|transformers|peft|bitsandbytes"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 39.4 MB/s eta 0:00:00
bitsandbytes                          0.4

## 5. Configure Weights & Biases (Optional)

In [4]:
# Set up wandb for experiment tracking
import os
import wandb

# Set API key directly
os.environ['WANDB_API_KEY'] = 'b29c0d2102aa226ead1fded36b786769f969d4f7'

# Initialize wandb
wandb.init(
    project="llm-testbench-gen",
    name="colab-training-run",
    config={
        "model": "TinyLlama-1.1B",
        "dataset": "AutoBench",
        "epochs": 30
    }
)

wandb: Currently logged in as: halgas-a (halgas-a-northeastern-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## 6. Update Config for Colab GPU

In [8]:
# Create optimized config for Colab GPU training with 24 clean examples
import yaml

# Load existing config
with open('configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Model settings - optimized for GPU
config['model']['quantization']['load_in_4bit'] = True

# Training parameters for small dataset (24 examples)
config['training']['per_device_train_batch_size'] = 1
config['training']['per_device_eval_batch_size'] = 1
config['training']['gradient_accumulation_steps'] = 4
config['training']['num_train_epochs'] = 30
config['training']['learning_rate'] = 5e-5
config['training']['warmup_steps'] = 20
config['training']['warmup_ratio'] = 0.1
config['training']['max_grad_norm'] = 0.5
config['training']['fp16'] = True

# LoRA configuration
config['model']['lora']['r'] = 16
config['model']['lora']['lora_alpha'] = 32
config['model']['lora']['lora_dropout'] = 0.05
config['model']['lora']['target_modules'] = ["q_proj", "v_proj", "k_proj", "o_proj"]

# Training schedule - FIXED: save_steps must be multiple of eval_steps
config['training']['logging_steps'] = 5
config['training']['eval_steps'] = 24        # Evaluate every epoch (24 examples / batch_size 1)
config['training']['save_steps'] = 48        # Save every 2 epochs (must be multiple of eval_steps)
config['training']['save_total_limit'] = 5
config['training']['load_best_model_at_end'] = True
config['training']['metric_for_best_model'] = 'eval_loss'
config['training']['greater_is_better'] = False

# Optimizer and scheduler
config['training']['optim'] = 'adamw_torch'
config['training']['lr_scheduler_type'] = 'constant_with_warmup'

# Generation settings
config['generation']['max_new_tokens'] = 4096
config['generation']['temperature'] = 0.7
config['generation']['top_p'] = 0.95
config['generation']['do_sample'] = True

# Data settings
config['data']['max_testbench_length'] = 4096

# Save updated config
with open('configs/config_colab.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False, sort_keys=False)

print("Config fixed and saved to config_colab.yaml")
print(f"eval_steps: {config['training']['eval_steps']} (every epoch)")
print(f"save_steps: {config['training']['save_steps']} (every 2 epochs - multiple of eval_steps)")

Config fixed and saved to config_colab.yaml
eval_steps: 24 (every epoch)
save_steps: 48 (every 2 epochs - multiple of eval_steps)


## 7. Run Training

In [9]:
# Fix the train.py file directly in Colab
!sed -i 's/evaluation_strategy=/eval_strategy=/g' /content/scripts/train.py

# Verify the change was made
!grep -n "eval_strategy" /content/scripts/train.py

159:        eval_strategy="steps",


In [10]:
# Set config file
!cp configs/config_colab.yaml configs/config.yaml

# Run training
!python scripts/train.py

2025-08-13 01:52:53.096066: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-13 01:52:53.113735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755049973.134846    3309 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755049973.141302    3309 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755049973.157900    3309 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## 8. Save Trained Model

In [11]:
# Compress the trained model
!cd models/checkpoints && zip -r /content/trained_model.zip * && cd /content
!ls -lh trained_model.zip

  adding: adapter_config.json (deflated 57%)
  adding: adapter_model.safetensors (deflated 8%)
  adding: chat_template.jinja (deflated 60%)
  adding: checkpoint-144/ (stored 0%)
  adding: checkpoint-144/tokenizer_config.json (deflated 69%)
  adding: checkpoint-144/adapter_model.safetensors (deflated 8%)
  adding: checkpoint-144/optimizer.pt (deflated 8%)
  adding: checkpoint-144/README.md (deflated 66%)
  adding: checkpoint-144/scaler.pt (deflated 60%)
  adding: checkpoint-144/trainer_state.json (deflated 79%)
  adding: checkpoint-144/tokenizer.model (deflated 55%)
  adding: checkpoint-144/adapter_config.json (deflated 57%)
  adding: checkpoint-144/rng_state.pth (deflated 25%)
  adding: checkpoint-144/special_tokens_map.json (deflated 73%)
  adding: checkpoint-144/chat_template.jinja (deflated 60%)
  adding: checkpoint-144/training_args.bin (deflated 51%)
  adding: checkpoint-144/scheduler.pt (deflated 57%)
  adding: checkpoint-144/tokenizer.json (deflated 85%)
  adding: checkpoint-180

In [12]:
# Download to local computer
from google.colab import files
files.download('trained_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 9. Install Icarus Verilog for Testing


In [13]:
# Install Icarus Verilog for compilation tests
!apt-get update
!apt-get install -y iverilog
!iverilog -version

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,929 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,111 kB]
Hit:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-

## 10. Run Evaluation

In [14]:
# Create the evaluation script if it doesn't exist
!cp /content/scripts/evaluate.py /content/scripts/evaluate_colab.py || echo "Creating new evaluation script"

# If evaluate.py doesn't exist, download it
if not os.path.exists('/content/scripts/evaluate.py'):
    print("Downloading evaluation script...")
    # You can paste the evaluate.py content here or upload it

In [15]:
# Run evaluation on the trained model
!python scripts/evaluate.py

# Check results
!ls -la data/test_results/

2025-08-13 02:04:38.992196: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-13 02:04:39.010420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755050679.032033    7133 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755050679.038578    7133 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755050679.055703    7133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [16]:
# Load and display evaluation results
import json

with open('data/test_results/evaluation_results.json', 'r') as f:
    results = json.load(f)

print("Evaluation Metrics:")
print("="*50)
for metric, value in results['metrics'].items():
    if isinstance(value, float):
        print(f"{metric}: {value:.2%}")
    else:
        print(f"{metric}: {value}")

Evaluation Metrics:
total_examples: 8
generation_success_rate: 100.00%
compilation_success_rate: 0.00%
simulation_success_rate: 0.00%
syntax_valid_rate: 0.00%
has_timescale_rate: 100.00%
has_module_rate: 100.00%
has_initial_rate: 100.00%
has_finish_rate: 50.00%
has_display_rate: 100.00%
avg_generation_length: 318437.50%
min_generation_length: 399
max_generation_length: 11115


In [20]:
cat data/test_results/generated_tb_3.v | tail -50

end
enddingpping,
endpinde. display display display anddcedcdnden for for forsscndtnd forgeddponeptneckdeneptdnentponcndeponepeptt
endnedding.block
end
endingen
endum.displayencingcentialxydottenningdponderpontytestdnreddedponencing:sl
ending.validssponnptdpondndnydpondtansspttedcding showdt
endd
endedptdyebd
endpadpoptdeddenponlrtentenponpocpadnmdpolbst
endeddpasspaddingdndenfttionpodedding_triggercecsleddeddeddedtedcirstpeptdingdingponponnsldpezzpoftledptfepept_shouldmingtingtingctyeburmingppcmm_valid, date_first_delay_display_pppsspon cptponponppedredentialmul
ended underningptential.portalnesp
endresetptcmm#logic delay commenting on display displayned show display display showninging (step or under showpping show. delayled.display tra potization (soling_first dis serdss ser c display sync (suppdffe wire block display;
flag_dedd_fixed ifd_sttion show and counter display timty display display countere simulation counterring `d `test wire_sim display timing for for display should timi

In [26]:
cat data/test_results/generated_tb_7.v

`timescale 1ns / 1ps module top_module_tb;
reg [7:0] a, b, c, d;
wire [7:0] min;
top_module uut ( .a(a), .b(b), .c(c), .d(d), .min(min) );
initial begin
a = 10;
b = 5;
c = 10;
d = 20;
$display("[TEST] %d %d %d %d %d", a, b, c, d, min);
#10;
$finish;
endinitial begin
#10;
endinitial begin
a = 11;
b = 6;
c = 11;
d = 21;
$display("[TEST] %d %d %d %d %d", a, b, c, d, min);
#10;
$finish;
end
endmodule

In [22]:
# View a sample generated testbench
import random

# List generated files
import glob
tb_files = glob.glob('data/test_results/generated_tb_*.v')

if tb_files:
    # Pick a random one to display
    sample_file = random.choice(tb_files)
    print(f"Viewing: {sample_file}")
    print("="*60)

    with open(sample_file, 'r') as f:
        print(f.read())
else:
    print("No generated testbenches found")

Viewing: data/test_results/generated_tb_3.v
`timescale 1ns / 1ps module top_module_tb;
reg clk, d, ar;
wire q;
top_module uut( .clk(clk), .d(d), .ar(ar), .q(q) );
initial begin
clk = 0;
ar = 0;
forever #10 clk = ~clk;
ar = 1;
$display("Testing ar %b, q=%d", ar, q);
ar = 0;
$display("Testing ar %b, q=%d", ar, q);
#100;
$finish;
end
endmodule `display;
q = 0;
// This is the original logic, wrongly indicated as a mutation for demonstration q = 1;
// No change q = 0;
// This is the original logic, wrongly indicated as a mutation for demonstration q = 1;
// This is the original logic, wrongly indicated as a mutation for demonstration q = 0;
// This is the original logic, wrongly indicated as a mutation for demonstration q = 1;
// This is the original logic, wrongly indicated as a mutation for demonstration q = 0;
// This is the original logic, wrongly indicated as a mutation for demonstration q = 1;
// This is the original logic, wrongly indicated as a mutation for demonstration q = 0;
// T

In [17]:
# Compress evaluation results
!cd data/test_results && zip -r /content/evaluation_results.zip * && cd /content

# Download
from google.colab import files
files.download('evaluation_results.zip')
print("Evaluation results downloaded!")

  adding: evaluation_results.json (deflated 81%)
  adding: generated_tb_0.v (deflated 50%)
  adding: generated_tb_1.v (deflated 43%)
  adding: generated_tb_2.v (deflated 72%)
  adding: generated_tb_3.v (deflated 49%)
  adding: generated_tb_4.v (deflated 72%)
  adding: generated_tb_5.v (deflated 44%)
  adding: generated_tb_6.v (deflated 47%)
  adding: generated_tb_7.v (deflated 67%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Evaluation results downloaded!


In [25]:
# Analyze training data quality
import json
import re

# Load training data
with open('data/processed/train/train.jsonl', 'r') as f:
    train_data = [json.loads(line) for line in f]

print(f"Total training examples: {len(train_data)}")

# Check for problematic patterns
issues = {
    'has_c_code': 0,
    'has_initial_brace': 0,
    'has_wrong_bit_width': 0,
    'has_mixed_comments': 0,
    'wire_assignment': 0
}

clean_examples = []
problematic_indices = []

for i, example in enumerate(train_data):
    tb = example['testbench_code']
    has_issue = False

    # Check for C code
    if 'int main' in tb or '#include' in tb:
        issues['has_c_code'] += 1
        has_issue = True

    # Check for wrong initial syntax
    if 'initial {' in tb:
        issues['has_initial_brace'] += 1
        has_issue = True

    # Check for wrong bit widths
    if re.search(r'\d{2,}\'b', tb):  # 10'b, 12'b, etc.
        issues['has_wrong_bit_width'] += 1
        has_issue = True

    # Check for mixed comments/documentation
    if '// Change:' in tb or '### Correctness:' in tb:
        issues['has_mixed_comments'] += 1
        has_issue = True

    # Check for wire assignments in initial blocks
    if re.search(r'initial.*?wire.*?=', tb, re.DOTALL):
        issues['wire_assignment'] += 1
        has_issue = True

    if has_issue:
        problematic_indices.append(i)
    else:
        clean_examples.append(example)

print("\nIssues found in training data:")
for issue, count in issues.items():
    print(f"  {issue}: {count} ({count/len(train_data)*100:.1f}%)")

print(f"\nClean examples: {len(clean_examples)} ({len(clean_examples)/len(train_data)*100:.1f}%)")
print(f"Problematic examples: {len(problematic_indices)}")

# Save clean training data
if clean_examples:
    with open('data/processed/train/train_clean.jsonl', 'w') as f:
        for example in clean_examples:
            f.write(json.dumps(example) + '\n')
    print(f"\nClean training data saved to train_clean.jsonl")

Total training examples: 24

Issues found in training data:
  has_c_code: 0 (0.0%)
  has_initial_brace: 0 (0.0%)
  has_wrong_bit_width: 0 (0.0%)
  has_mixed_comments: 0 (0.0%)
  wire_assignment: 0 (0.0%)

Clean examples: 24 (100.0%)
Problematic examples: 0

Clean training data saved to train_clean.jsonl
